In [1]:
import numpy as np
import pandas as pd
import os
import re
# from library.sb_utils import save_file



In [2]:
pd.set_option("display.max_rows", 2000)

In [3]:
df = pd.read_excel('../data/raw/online_retail_II.xlsx', sheet_name = 'Year 2010-2011')

In [4]:
df.head(5)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      541910 non-null  object        
 1   StockCode    541910 non-null  object        
 2   Description  540456 non-null  object        
 3   Quantity     541910 non-null  int64         
 4   InvoiceDate  541910 non-null  datetime64[ns]
 5   Price        541910 non-null  float64       
 6   Customer ID  406830 non-null  float64       
 7   Country      541910 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [6]:
#First, rename the columns by removing white spaces in the columns names
df = df.rename(columns={'Customer ID' : 'CustomerID'})
df.columns

Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'CustomerID', 'Country'],
      dtype='object')

In [7]:
df.describe()

,Quantity,InvoiceDate,Price,CustomerID
count,541910.000000,541910,541910.000000,406830.000000
mean,9.552234,2011-07-04 13:35:22.342307584,4.611138,15287.684160
min,-80995.000000,2010-12-01 08:26:00,-11062.060000,12346.000000
25%,1.000000,2011-03-28 11:34:00,1.250000,13953.000000
50%,3.000000,2011-07-19 17:17:00,2.080000,15152.000000
75%,10.000000,2011-10-19 11:27:00,4.130000,16791.000000
max,80995.000000,2011-12-09 12:50:00,38970.000000,18287.000000
std,218.080957,NaN,96.759765,1713.603074


<font color='red'>Inconsistency Found:</font> Customer ID count is not the same as that of the Quantity


In [8]:
#Find ratio of Null Customer ID
ratio_null_cust_id = round(len(df.loc[df['CustomerID'].isna()]) / len(df) * 100, 2)
ratio_null_cust_id

24.93

- Approximately 25% of Customer ID is (null). 
- As the focus of our model is on Customer Segmentation, (null) Customer ID is of no value to the analysis; therefore, it is imperative to drop the (null) values.

In [9]:
# Drop null rows in Customer id
df = df.dropna(subset = ['CustomerID'])
df.isna().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
CustomerID     0
Country        0
dtype: int64

In [10]:
df.head(5)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


<font color='red'>Inconsitency Found:</font> Customer ID is of type float, and the decimals removed because according to the documentation: 

- CustomerID: Customer number. Nominal. A 5-digit integral number uniquely assigned to each customer. 
The decimals in this column should be removed

In [11]:
#Convert Customer ID from Float to int
df['CustomerID'] = df['CustomerID'].astype(int)
df.head(5)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


In [12]:
df.describe()

,Quantity,InvoiceDate,Price,CustomerID
count,406830.000000,406830,406830.000000,406830.000000
mean,12.061276,2011-07-10 16:31:30.127424512,3.460507,15287.684160
min,-80995.000000,2010-12-01 08:26:00,0.000000,12346.000000
25%,2.000000,2011-04-06 15:02:00,1.250000,13953.000000
50%,5.000000,2011-07-31 11:48:00,1.950000,15152.000000
75%,12.000000,2011-10-20 13:06:00,3.750000,16791.000000
max,80995.000000,2011-12-09 12:50:00,38970.000000,18287.000000
std,248.693065,NaN,69.315080,1713.603074


<font color='red'>Inconsistencies Found: </font> Minimum Value for Quantity and Price is negative


In [13]:
#Inspect the negative quantities
df.loc[df['Quantity'] < 0].head(5)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom


Compare negative values. Check if all these negative quantities are from Cancelled Invoices ( Invoices with "C" as a prefix )

In [14]:
#Compare negative values
df['Invoice'] = df['Invoice'].astype(str)
cancelled = df['Invoice'].str.contains(r'^[Cc]') 
negative = df['Quantity'] < 0
positive = df['Quantity'] >= 0

print(f'Negative quantities : {len(df.loc[df['Quantity'] < 0])}')
print(f'Cancelled Invoices: {len(df[cancelled])}')
print(f'Negative quantities with Cancalled invoices: {len(df[cancelled & ~positive])}' )
print(f'Positive quantities with Cancelled invoices: {len(df[cancelled & positive])}' )

Negative quantities : 8905
Cancelled Invoices: 8905
Negative quantities with Cancalled invoices: 8905
Positive quantities with Cancelled invoices: 0


Findings: All negative quantities are from Cancelled Invoices. For this analysis, cancelled invoices will be discarded.

In [15]:
df = df[df['Quantity'] > 0]

print(len(df))
# print(len(df.loc[df['Quantity'] < 0].head(5)))
df.head(10)

397925


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850,United Kingdom
9,536368,22960,JAM MAKING SET WITH JARS,6,2010-12-01 08:34:00,4.25,13047,United Kingdom


In [16]:
df.describe()

,Quantity,InvoiceDate,Price,CustomerID
count,397925.000000,397925,397925.000000,397925.000000
mean,13.021793,2011-07-10 23:44:09.817126400,3.116212,15294.308601
min,1.000000,2010-12-01 08:26:00,0.000000,12346.000000
25%,2.000000,2011-04-07 11:12:00,1.250000,13969.000000
50%,6.000000,2011-07-31 14:39:00,1.950000,15159.000000
75%,12.000000,2011-10-20 14:33:00,3.750000,16795.000000
max,80995.000000,2011-12-09 12:50:00,8142.750000,18287.000000
std,180.419984,NaN,22.096773,1713.172738


In [17]:
df.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
CustomerID     0
Country        0
dtype: int64

## Inspecting Other Columns

Check Invoices column. Per the description:

- InvoiceNo: Invoice number. Nominal. A 6-digit integral number uniquely assigned to each transaction. If this code starts with the letter 'c', it indicates a cancellation. 

In [29]:
#Check for incorrect invoices 
df['Invoice'] = df['Invoice'].astype(str)
incinv = df[df['Invoice'].str.match(r'^\d{5}$')]
incinv

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,CustomerID,Country



Check Countries for misspelled entries or inconsistencies

In [30]:
#Inspect the countries
df['Country'].unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Finland',
       'Austria', 'Greece', 'Singapore', 'Lebanon',
       'United Arab Emirates', 'Israel', 'Saudi Arabia', 'Czech Republic',
       'Canada', 'Unspecified', 'Brazil', 'USA', 'European Community',
       'Bahrain', 'Malta', 'RSA'], dtype=object)

Check Stock Codes. Per the description: 
- StockCode: Product (item) code. Nominal. A 5-digit integral number uniquely assigned to each distinct product. 

In [31]:
#Stock Codes with non-numeric codes
df['StockCode'] = df['StockCode'].astype(str)
notnum = df[df['StockCode'].str.match(r'^\D')]
notnum['StockCode'].value_counts()
notnum.groupby(['StockCode', 'Description']).agg({'StockCode': 'count'}).head(10)

,,StockCode
StockCode,Description,
BANK CHARGES,Bank Charges,12
C2,CARRIAGE,133
DOT,DOTCOM POSTAGE,16
M,Manual,290
PADS,PADS TO MATCH ALL CUSHIONS,4
POST,POSTAGE,1100


Inspect the other values of the non-conforming stock codes for inconsitencies or irregularities

In [32]:
#Stock Codes with numeric codes followed by Characters
alphanum = df[df['StockCode'].str.match(r'\d{5}\D')]
alphanum.groupby(['StockCode', 'Description']).agg({'StockCode': 'count'}).head(10)

,,StockCode
StockCode,Description,
10123C,HEARTS WRAPPING TAPE,3
10124A,SPOTS ON RED BOOKCOVER TAPE,5
10124G,ARMY CAMO BOOKCOVER TAPE,4
15044A,PINK PAPER PARASOL,79
15044B,BLUE PAPER PARASOL,48
15044C,PURPLE PAPER PARASOL,61
15044D,RED PAPER PARASOL,74
15056BL,EDWARDIAN PARASOL BLACK,291
15056N,EDWARDIAN PARASOL NATURAL,382


In [33]:
print(len(notnum))
print(len(alphanum))
nonconf = pd.concat([notnum, alphanum])
nonconf

1555
33251


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,CustomerID,Country
45,536370,POST,POSTAGE,3,2010-12-01 08:45:00,18.00,12583,France
386,536403,POST,POSTAGE,1,2010-12-01 11:27:00,15.00,12791,Netherlands
1123,536527,POST,POSTAGE,1,2010-12-01 13:04:00,18.00,12662,Germany
1423,536540,C2,CARRIAGE,1,2010-12-01 14:05:00,50.00,14911,EIRE
2239,536569,M,Manual,1,2010-12-01 15:35:00,1.25,16274,United Kingdom
...,...,...,...,...,...,...,...,...
541778,581578,84997C,CHILDRENS CUTLERY POLKADOT BLUE,8,2011-12-09 12:16:00,4.15,12713,Germany
541809,581579,85099C,JUMBO BAG BAROQUE BLACK WHITE,10,2011-12-09 12:19:00,1.79,17581,United Kingdom
541841,581580,84993A,75 GREEN PETIT FOUR CASES,2,2011-12-09 12:20:00,0.42,12748,United Kingdom
541847,581580,85049A,TRADITIONAL CHRISTMAS RIBBONS,1,2011-12-09 12:20:00,1.25,12748,United Kingdom


In [34]:
#Inspect the quantities
nonconf.loc[df['Quantity'] < 0]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,CustomerID,Country


In [35]:
#Inspect the countries
nonconf['Country'].unique()

array(['France', 'Netherlands', 'Germany', 'EIRE', 'United Kingdom',
       'Switzerland', 'Portugal', 'Italy', 'Belgium', 'Channel Islands',
       'Denmark', 'Spain', 'Finland', 'Austria', 'Sweden', 'Norway',
       'Cyprus', 'Australia', 'Singapore', 'Greece', 'European Community',
       'Poland', 'Malta', 'Canada', 'United Arab Emirates',
       'Czech Republic', 'RSA', 'Lithuania', 'Japan', 'Iceland',
       'Lebanon', 'Brazil', 'USA', 'Bahrain', 'Unspecified', 'Israel'],
      dtype=object)

We will assume that all the non-conforming stock codes are valid transactions done manually or automatically but outside the order placement process. This is because other columns for the non-conforming stock codes have no inconsistent values.

Create a new month column from Invoice Date (for later use)

In [127]:
df['Month'] = df['InvoiceDate'].dt.month
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,CustomerID,Country,Month
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,12
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,12
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,12
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,12
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,12
...,...,...,...,...,...,...,...,...,...
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France,12
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France,12
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France,12
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680,France,12


Check the values under the new 'Month' column

In [128]:
#Inspect no. of monthly transactions
df['Month'].value_counts().sort_index()

Month
1     21232
2     19928
3     27177
4     22644
5     28322
6     27185
7     26827
8     27013
9     40030
10    49557
11    64545
12    43465
Name: count, dtype: int64

Save Cleaned File

In [22]:
with pd.ExcelWriter("../data/interim/online_retail_cleaned.xlsx", engine='openpyxl') as writer:
    # df.to_excel(writer, sheet_name="2010-2011", index=False)